In [1]:
from itertools import combinations, combinations_with_replacement, product
import random

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from Bio.SeqIO import parse

from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix

from ushuffle import shuffle as kmer_shuffle

from regex_classifier import RegexGeneticEnsembleClassifier

In [8]:
%%bash
source activate jupyter_python3
bedtools intersect -s -a K_called_peaks.bed -b PDS_called_peaks.bed > KPDS_called_peaks.bed
bedtools getfasta -s -fi hg19.fa -bed KPDS_called_peaks.bed -fo stdout | \
  fold -w 80 > training_seqs.fa

In [10]:
# create the components for regular expression
bases = []
weights = []
for i in (1, 2, 3, 4):
    b = ['[' + ''.join(b) + ']' for b in combinations('ACGT', i)]
    bases += b
    weights += [np.reciprocal(float(i)), ] * len(b)
sizes = ['{{{:d},{:d}}}?'.format(x, y) if x != y
         else '{{{:d}}}'.format(x)
         for x, y in combinations_with_replacement(range(1, 4), 2)]
components, component_weights = zip(
    *[(b + s, w) for (b, w), s in product(zip(bases, weights), sizes)])
# add empty string
components = list(components) + ['', ]
component_weights = np.asarray(list(component_weights) + [1, ])
component_weights /= component_weights.sum()

In [23]:
training_seqs = []
labels = []
with open('./training_seqs.fa') as f:
    for record in parse(f, 'fasta'):
        labels.append(1)
        seq = str(record.seq).upper()
        training_seqs.append(seq)
        labels.append(0)
        training_seqs.append(kmer_shuffle(seq, len(seq), 1))
zipped = list(zip(labels, training_seqs))
random.shuffle(zipped)
labels, training_seqs = zip(*zipped)

In [26]:
r = RegexGeneticEnsembleClassifier(
    regex_components=components,
    component_weight=component_weights,
    n_pops=30, pop_size=1000, n_gen=2000, n_best=1,
    ind_len=15, n_selected=500, n_offspring=200,
    batch_size=100, early_stopping=5,
    mut_rate=2, crossover_rate=2)

In [ ]:
r.fit(training_seqs, labels)

In [ ]:
r.score(training_seqs[-1000:], labels[-1000:])

In [ ]:
roc_auc_score(labels[-1000:], r.predict_proba(training_seqs[-1000:])[:, 1])

In [ ]:
fpr, tpr, _ = roc_curve(labels[-1000:],
                        r.predict_proba(training_seqs[-1000:])[:, 1])
plt.plot(fpr, tpr)
plt.show()

In [ ]:
r.estimators_